<a href="https://colab.research.google.com/github/kkkkkkkm/Torch/blob/main/RNN/char_seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch

In [2]:
sample = "if you want you"

In [3]:
char_set = list(set(sample)) #중복된 character 제거, unique character list 생성
print(char_set)

[' ', 'i', 'u', 'a', 'n', 'f', 'o', 'y', 'w', 't']


In [4]:
char_dict = {c: i for i, c in enumerate(char_set)} #enumerate를 사용하여 dictionary 생성 character와 index 할당
print(char_dict)

{' ': 0, 'i': 1, 'u': 2, 'a': 3, 'n': 4, 'f': 5, 'o': 6, 'y': 7, 'w': 8, 't': 9}


In [5]:
dict_size = len(char_dict)
hidden_size = len(char_dict)
learning_rate = 0.1

print(f" dict_size = {dict_size}, hidden_size = {hidden_size}")

 dict_size = 10, hidden_size = 10


In [6]:
sample_idx = [char_dict[i] for i in sample] #만든 vocabulary에 sample의 글자를 넣어서 sample 인덱스 값 추출
print(sample_idx)

[1, 5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]


In [7]:
x_data = [sample_idx[:-1]]
print(x_data)
y_data = [sample_idx[1:]]
print(y_data)

[[1, 5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6]]
[[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]]


In [8]:
x_one_hot = [np.eye(dict_size)[x] for x in x_data] #크기가 dict_size인 identity matrix 생성 후 [x]번째 행을 가져와서 list에 담음
print(x_one_hot)

[array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])]


In [9]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print(f' X = {X}, Y = {Y}')
print(X.shape)

 X = tensor([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]]]), Y = tensor([[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]])
torch.Size([1, 14, 10])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


In [10]:
rnn = torch.nn.RNN(10, hidden_size, batch_first= True)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), learning_rate)

In [15]:
for i in range(100):
  optimizer.zero_grad()

  outputs, _state = rnn(X)

  loss = criterion(outputs.view(-1, 10), Y.view(-1))
  loss.backward() #가중치에 대한 현재 기울기가 양수인지 음수인지.
  
  optimizer.step()#가중치 업데이트

  result = outputs.data.argmax(axis = 2) 

  result_str = ''.join([char_set[i] for i in np.squeeze(result)])

  print(i , "loss :", loss.item(), "prediction:", result, "true Y", y_data, "prediction str: ", result_str)

0 loss : 0.8077933192253113 prediction: tensor([[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]]) true Y [[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]] prediction str:  f you want you
1 loss : 0.8077300786972046 prediction: tensor([[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]]) true Y [[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]] prediction str:  f you want you
2 loss : 0.8076682090759277 prediction: tensor([[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]]) true Y [[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]] prediction str:  f you want you
3 loss : 0.8076070547103882 prediction: tensor([[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]]) true Y [[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]] prediction str:  f you want you
4 loss : 0.8075475096702576 prediction: tensor([[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]]) true Y [[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]] prediction str:  f you want you
5 loss : 0.8074884414672852 prediction: tensor([[5, 0, 7, 6, 2, 0, 8, 3, 4, 9, 0, 7, 6, 2]]) true Y [[5, 0, 7, 6, 2

In [62]:
test_input = torch.Tensor(1,14,10)
print(test_input.shape)


test_out, T_state = rnn(X)
print(test_out.shape)

torch.Size([1, 14, 10])
torch.Size([1, 14, 10])


In [72]:
a = test_out.data.argmax(axis = 2)